# Medical image steganography

In [ ]:
import cv2
import numpy as np

### Embedding process

In [ ]:
def embed_watermark(image, watermark_text, position=(10, 30), font_scale=0.5):
    watermarked_image = image.copy()
    cv2.putText(watermarked_image, watermark_text, position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), 1, cv2.LINE_AA)
    return watermarked_image

def embed_secret_message_with_delimiter(image, secret_message):
    secret_message_with_delimiter = secret_message + "####"
    binary_message = ''.join(format(ord(char), '08b') for char in secret_message_with_delimiter)

    flat_image = image.flatten()

    for i in range(len(binary_message)):
        flat_image[i] = (flat_image[i] & ~1) | int(binary_message[i])

    return flat_image.reshape(image.shape)

### Extraction

In [ ]:
def extract_secret_message_until_delimiter(image, password, correct_password):
    if password != correct_password:
        return "Incorrect password! Access denied."

    flat_image = image.flatten()
    binary_message = ''

    for i in range(len(flat_image)):
        binary_message += str(flat_image[i] & 1)

        if len(binary_message) % 8 == 0:
            char = chr(int(binary_message[-8:], 2))

            if binary_message.endswith('00100011001000110010001100100011'):
                break

    extracted_message = ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message)-32, 8))

    return extracted_message

### Region Of Interest highlighter

In [ ]:
def highlight_roi(image, roi_coords):
    roi_image = image.copy()
    for (x, y, w, h) in roi_coords:
        cv2.rectangle(roi_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return roi_image

def save_roi_image(image, roi_coords, output_roi_image_path):
    roi_image = highlight_roi(image, roi_coords)
    cv2.imwrite(output_roi_image_path, roi_image)
    print(f"ROI image saved at: {output_roi_image_path}")

### Embedding function

In [ ]:
def combine_watermark_and_message(image_path, secret_message, watermark_text, output_image_path, output_roi_image_path):
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Could not load image from {image_path}")
        return None

    roi_coords = [(0, 0, 400, 100)]
    save_roi_image(image, roi_coords, output_roi_image_path)

    watermarked_image = embed_watermark(image, watermark_text)

    height, width = image.shape[:2]

    mask = np.ones((height, width), dtype=np.uint8)

    for (x, y, w, h) in roi_coords:
        mask[y:y + h, x:x + w] = 0
    message_image = watermarked_image.copy()
    if np.count_nonzero(mask) > len(secret_message) * 8:
        message_image = embed_secret_message_with_delimiter(message_image, secret_message)
    else:
        print("Not enough space to embed the secret message in the available area.")

    cv2.imwrite(output_image_path, message_image)
    print(f"Image saved with watermark and secret message at: {output_image_path}")

    return message_image

Details give for execution

In [ ]:
image_path = 'medical_image.png'
secret_message = "Patient Name: John Doe, Disease: XYZ, Doctor: James"
watermark_text = "Watermark"
output_image_path = 'output_image.png'
output_roi_image_path = 'roi_image.png'
correct_password = input("Enter a password to protect: ")

message_image = combine_watermark_and_message(image_path, secret_message, watermark_text, output_image_path, output_roi_image_path)

Enter a password to protect: 1234
ROI image saved at: roi_image.png
Image saved with watermark and secret message at: output_image.png


Extracting the secret message which is password protected

In [ ]:
password = input("Enter the password to access the secret message: ")

extracted_message = extract_secret_message_until_delimiter(cv2.imread(output_image_path), password, correct_password)
print(f"Extracted secret message: {extracted_message}")

Enter the password to access the secret message: 1234
Extracted secret message: Patient Name: John Doe, Disease: XYZ, Doctor: James
